In [2]:
import pickle, json, requests, csv, copy, os, random
from json import JSONDecodeError
from text_to_uri import *
import numpy as np
import pprint as pp
import urllib.request, urllib.parse
import utils

In [26]:
class_uri = ['/c/en/atheism',
            '/c/en/graphics',
            '/c/en/operating_system',
            '/c/en/ibm',
            '/c/en/mac',
            '/c/en/windows',
            '/c/en/sale',
            '/c/en/auto',
            '/c/en/motorcycle',
            '/c/en/baseball',
            '/c/en/hockey',
            '/c/en/crypt',
            '/c/en/electronics',
            '/c/en/medical',
            '/c/en/space',
            '/c/en/christian',
            '/c/en/gun',
            '/c/en/middle_east',
            '/c/en/politics',
            '/c/en/religion']

In [59]:
class_uri = ['/c/en/company',
            '/c/en/education',
            '/c/en/artist',
            '/c/en/athlete',
            '/c/en/officer',
            '/c/en/transport',
            '/c/en/building',
            '/c/en/nature',
            '/c/en/village',
            '/c/en/animal',
            '/c/en/plant',
            '/c/en/album',
            '/c/en/film',
            '/c/en/writing']

In [10]:
class_uri = ['/c/en/monitor',
             '/c/en/watch',
             '/c/en/blu_ray',
             '/c/en/cd_player',
             '/c/en/network',
             '/c/en/battery',
             '/c/en/graphics_card',
             '/c/en/xbox',
             '/c/en/telephone',
             '/c/en/car_stereo',
             '/c/en/video_recorder',
             '/c/en/alarm_clock',
             '/c/en/camera',
             '/c/en/camcorder',
             '/c/en/hard_drive',
             '/c/en/home_theater',
             '/c/en/subwoofer',
             '/c/en/rice_cooker',
             '/c/en/webcam',
             '/c/en/kindle',
             '/c/en/scanner',
             '/c/en/radar_detector',
             '/c/en/vacuum',
             '/c/en/cable_modem',
             '/c/en/cellphone',
             '/c/en/wireless_router',
             '/c/en/printer',
             '/c/en/fan',
             '/c/en/amplifier',
             '/c/en/microwave',
             '/c/en/gps',
             '/c/en/microphone',
             '/c/en/tv',
             '/c/en/projector',
             '/c/en/mouse',
             '/c/en/recorder',
             '/c/en/memory_card',
             '/c/en/iron',
             '/c/en/laptop',
             '/c/en/headphone',
             '/c/en/video',
             '/c/en/mp3_player',
             '/c/en/keyboard',
             '/c/en/tablet',
             '/c/en/media_player',
             '/c/en/lamp',
             '/c/en/remote_control',
             '/c/en/speaker',
             '/c/en/computer',
             '/c/en/dvd']

In [4]:
def random_seen_unseen(class_uri, random_unseen_percentage):
    num_class = len(class_uri)
    num_unseen_class = int(num_class * random_unseen_percentage)
    class_id_list = list(range(num_class))
    random.shuffle(class_id_list)
    unseen_class = class_id_list[:num_unseen_class]
    seen_class = class_id_list[num_unseen_class:]
    return sorted(seen_class), sorted(unseen_class)

In [5]:
def read_test_file(filename):
    with open(filename, encoding = 'utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=',')
        test = []
        for row in reader:
#             test.append({'id': row['Unnamed: 0'], 'class': int(row['class'])-1, 'text': row['selected']})
            test.append({'id': row[''], 'class': int(row['class'])-1, 'text': row['text']})
    return test

In [6]:
def predict_by_class_labels(text, class_label):
    class_label_for_match = [' ' + c.replace('_', ' ') + ' ' for c in class_label]
    count = [(i, text.count(class_label_for_match[i])) for i in range(len(class_label_for_match))]
    max_count = max([c[1] for c in count])
    max_class = [c[0] for c in count if c[1] == max_count]
    return random.choice(max_class) 

In [7]:
def ans_index_to_matrix(class_uri, list_of_index):
    matrix = np.zeros((len(list_of_index), len(class_uri)))
    for i in range(len(matrix)):
        matrix[i][list_of_index[i]] = 1
    return matrix

In [8]:
def average_measures(stats_list):
    stats = dict()
    keys = stats_list[0].keys()
    for mea in keys:
        mea_list = [s[mea] for s in stats_list]
        stats[mea] = np.mean(mea_list)
    return stats

In [9]:
def classify_count_baseline(test, class_uri):
    stats_list = []
    stats_seen_list = []
    stats_unseen_list = []
    
    for n_round in range(10):
        seen_class, unseen_class = random_seen_unseen(class_uri, random_unseen_percentage = 0.25)
        class_label = [c[6:] for c in class_uri]

        seen_test = [x for x in test if x['class'] in seen_class]
        unseen_test = [x for x in test if x['class'] in unseen_class]

        seen_predicted = [predict_by_class_labels(x['text'], class_label) for x in seen_test]
        unseen_predicted = [predict_by_class_labels(x['text'], class_label) for x in unseen_test]

        seen_gt = [x['class'] for x in seen_test]
        unseen_gt = [x['class'] for x in unseen_test]

        seen_predicted_matrix = ans_index_to_matrix(class_uri, seen_predicted)
        unseen_predicted_matrix = ans_index_to_matrix(class_uri, unseen_predicted)
        seen_gt_matrix = ans_index_to_matrix(class_uri, seen_gt)
        unseen_gt_matrix = ans_index_to_matrix(class_uri, unseen_gt)

        stats = utils.get_statistics(np.concatenate((seen_predicted_matrix, unseen_predicted_matrix), axis = 0), np.concatenate((seen_gt_matrix, unseen_gt_matrix), axis = 0), True)
        stats_seen = utils.get_statistics(seen_predicted_matrix, seen_gt_matrix, True)
        stats_unseen = utils.get_statistics(unseen_predicted_matrix, unseen_gt_matrix , True)
        
        stats_list.append(stats)
        stats_seen_list.append(stats_seen)
        stats_unseen_list.append(stats_unseen)
        
        try:
            print('Round:', n_round)
            print('unseen:', unseen_class)
            print('seen:', seen_class)

            print("uns count baseline:  %s" % (utils.dict_to_string_4_print(stats_unseen)))
            print("see count baseline:  %s" % (utils.dict_to_string_4_print(stats_seen)))
            print("all count baseline:  %s" % (utils.dict_to_string_4_print(stats)))
        except:
            print("error")
        print('------------------------------------------------------------------------')

    stats = average_measures(stats_list)
    stats_seen = average_measures(stats_seen_list)
    stats_unseen = average_measures(stats_unseen_list)
    
    try:
        print('On average')
        print("uns count baseline:  %s" % (utils.dict_to_string_4_print(stats_unseen)))
        print("see count baseline:  %s" % (utils.dict_to_string_4_print(stats_seen)))
        print("all count baseline:  %s" % (utils.dict_to_string_4_print(stats)))
    except:
        print("error")
    print('------------------------------------------------------------------------')

In [11]:
classify_count_baseline(test, class_uri)

Round: 0
unseen: [9, 12, 13, 16, 20, 25, 26, 33, 38, 42, 46, 47]
seen: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 14, 15, 17, 18, 19, 21, 22, 23, 24, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 39, 40, 41, 43, 44, 45, 48, 49]
uns count baseline:  accuracy: 0.974, macro-F1: 0.213, macro-precision: 0.151, macro-recall: 0.362, micro-F1: 0.362, micro-precision: 0.362, micro-recall: 0.362, single-label-error: 0.638
see count baseline:  accuracy: 0.971, macro-F1: 0.243, macro-precision: 0.225, macro-recall: 0.264, micro-F1: 0.264, micro-precision: 0.264, micro-recall: 0.264, single-label-error: 0.736
all count baseline:  accuracy: 0.972, macro-F1: 0.278, macro-precision: 0.269, macro-recall: 0.288, micro-F1: 0.288, micro-precision: 0.288, micro-recall: 0.288, single-label-error: 0.712
------------------------------------------------------------------------
Round: 1
unseen: [1, 5, 13, 17, 20, 24, 26, 29, 31, 33, 36, 44]
seen: [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 19, 21, 22, 23, 25, 27, 2

In [21]:
print(random_seen_unseen(class_uri, 0.25))

([0, 2, 3, 4, 5, 6, 7, 10, 11, 13, 14, 15, 16, 17, 18], [1, 8, 9, 12, 19])


In [22]:
class_label = [c[6:] for c in class_uri]
print(class_label)

['atheism', 'graphics', 'operating_system', 'ibm', 'mac', 'windows', 'sale', 'auto', 'motorcycle', 'baseball', 'hockey', 'crypt', 'electronics', 'medical', 'space', 'christian', 'gun', 'middle_east', 'politics', 'religion']


In [31]:
test = read_test_file('../data/newsgroup/test.csv')
# print(test[0]['class'], predict_by_class_labels(test[0]['text'], class_label))

In [58]:
test = read_test_file('../data/dbpedia/test.csv')

In [3]:
test = read_test_file('../data/review/test.csv')